In [2]:
import gui  

text = gui.text
print(text)

pygame 2.1.2 (SDL 2.0.18, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html
aaaaaaaaaa


# Coref resolution

In [3]:
import pickle

# load saved model
with open(r'coref_resolution\allen_coref' , 'rb') as f:
    predictor = pickle.load(f)



text = """A cat is walking on a snowy day. It jumped over a stone. It died. Dog is walking in the opposite direction. It ran. The dog said "Hello world, the cat is going to die hahaha"."""
# text = """Once there lived a crow. He wished to be colorful and beautiful like other birds. 
# He then went to the parrot and shared his thoughts. 
# But parrot said "Peacock is most beautiful bird so talk to him". 
# Then the crow went to the peacock and told him about his looks. 
# Then the peacock replied, "You are the luckiest bird that has been never caged in life and we because of our beauty stay caged, and you are always free."
# After listening to this, crow realized his mistake and thanked God for making him like this and he flew away happily."""

weathers = ["sunny", "rainy", "windy", "snowy", "night"]

# Default weather
extracted_weather = "sunny"

for weather in weathers:
    if weather in text:
        extracted_weather = weather
        print(extracted_weather)
        break

prediction = predictor.predict(document=text)  # get prediction
print("Clsuters:-")
for cluster in prediction['clusters']:
    print(cluster)  # list of clusters (the indices of spaCy tokens)
print('\n\n') #Newline

coref_text = predictor.coref_resolved(text)

print('Coref resolved: ', coref_text)  # resolved text

c:\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


snowy
Clsuters:-
[[0, 1], [9, 9], [15, 15], [36, 37]]
[[18, 18], [26, 26], [29, 30]]



Coref resolved:  A cat is walking on a snowy day. A cat jumped over a stone. A cat died. Dog is walking in the opposite direction. Dog ran. Dog said "Hello world, A cat is going to die hahaha".


# SAO parsing

In [4]:
import re
from clausIE import svo

# coref_text = """A cat walks rapidly on a sunny day. A cat fell over a stone. A cat died. Dog said "Hello World"."""
SVOs = svo.extract_svo(text, coref_text)

print(SVOs)

C:\Users\shour\AppData\Roaming\Python\Python39\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.4.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


---------------SVO MODULE---------------
[<SV, A cat, is walking, None, None, None, [on a snowy day]>]
A cat is walking on a snowy day
else
A cat is walking on a snowy day
[('A cat', 'is walking')]
[<SV,  A cat, jumped, None, None, None, [over a stone]>]
 A cat jumped over a stone
else
 A cat jumped over a stone
[(' A cat', 'jumped')]
[<SV,  A cat, died, None, None, None, []>]
 A cat died
else
 A cat died
[(' A cat', 'died')]
[<SV, Dog, is walking, None, None, None, [in the opposite direction]>]
 Dog is walking in the opposite direction
else
 Dog is walking in the opposite direction
[('Dog', 'is walking')]
[<SV, Dog, ran, None, None, None, []>]
 Dog ran
else
 Dog ran
[('Dog', 'ran')]
[<SVOC,  Dog, said, None, Hello world, A cat is going to die hahaha, []>, <SVC, A cat, is going, None, None, to die hahaha, []>]
 Dog said "Hello world, A cat is going to die hahaha"
in if
 Dog said "Hello world, A cat is going to die hahaha"
[(' Dog', 'said', 'Hello world, the cat is going to die hahaha')

In [5]:
from nltk.corpus import stopwords
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
# nltk.download('omw-1.4')
# nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
svos = list()
characters = set()

def filter_sent(text):
    word_tokens = word_tokenize(text)
    filtered_subject = [w.lower() for w in word_tokens if not w.lower() in stop_words]
    return "".join(filtered_subject)

for svo in SVOs:
    raw_subject = svo[0]

    # Tokenize subject

    # Remove stop words from subject
    subject = filter_sent(svo[0])
    characters.add(subject)

    final_svo = ()

    if svo[1] == "said":
        # If dialogue exists, append both the verb 'said' and dialogue
        action = WordNetLemmatizer().lemmatize(filter_sent(svo[1]), 'v')
        dialogue = svo[2]
        final_svo = (subject, action, dialogue)
    else:
        # If no dialogue, just append the verb
        action = WordNetLemmatizer().lemmatize(filter_sent(svo[1]), 'v')
        final_svo = (subject, action)

    svos.append(final_svo)

print(svos)
print(characters)

[('cat', 'walk'), ('cat', 'jump'), ('cat', 'die'), ('dog', 'walk'), ('dog', 'run'), ('dog', 'say', 'Hello world, the cat is going to die hahaha')]
{'cat', 'dog'}


In [6]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

actions_movement = {'die' : 0, 'fall' : 0, 'hurt' : 0, 'idle' : 0, 'jump' : 1, 'run' : 1, 'slide' : 1, 'walk' : 1, 'say': 0}
#word2vec similarity between the incoming action vs the ones we have in list of actions and then set a threshold and execute the action based on it

In [16]:
from numpy import character
import pygame
import glob
from time import sleep
import audioread

from gtts import gTTS

language = 'en'

SIZE = WIDTH, HEIGHT = 1280, 720 #the width and height of our screen
FPS = 7 #Frames per second

screen = pygame.display.set_mode(SIZE)
bg = pygame.transform.scale(pygame.image.load('assets\\weather\\' + extracted_weather + '.jpg') , SIZE)

dir_list = {'r' : 200, 'l' : 400} #moving towards which direction

dialogues = []
dialogue_count = 0
 
class MySprite(pygame.sprite.Sprite):

    def __init__(self, char, x, y, dir):
        super(MySprite, self).__init__()
        
        self.images = {}
        self.actions = []
        # self.images = [pygame.image.load(img) for img in glob.glob("pygame\\cat\\*.png")]
        # for action in svos[char]:
        #     if type(action) == list:
        #         dialogues.append(action[1])
        #     action = WordNetLemmatizer().lemmatize(action[0] if type(action) == list else action,'v')   
        #     self.actions.append(action)
        #     print(self.actions)
            
        #     if action not in self.images:
        #         self.images[action] = [pygame.transform.scale(pygame.image.load(img) , (200,200)) for img in glob.glob("assets\\characters" + char + "\\" + action + "\\*.png")]
            
        # print(self.images)
        self.char = char
        self.index = 0
        self.action_count = 0
        self.dir = dir
        self.x = x
        self.y = y
        self.prev_action = None  

    def play_dialogue(self, dialogue):
        myobj = gTTS(text=dialogue, lang=language, slow=False)
        myobj.save("dialogue.mp3")
        with audioread.audio_open('dialogue.mp3') as f:
            totalsec = f.duration

        pygame.mixer.init()
        pygame.mixer.music.load("dialogue.mp3")
        pygame.mixer.music.set_volume(1)
        pygame.mixer.music.play()
        sleep(totalsec)
        
        pygame.mixer.music.unload()

        
    def movement_update(self, fps = FPS):
        # if self.x == 880:
        #     self.dir = 'l'
        # elif self.x == 0:
        #     self.dir = 'r'
        if self.dir == 'r':                        
            self.x += fps
        else:
            self.image = pygame.transform.flip(self.image, True, False)
            self.x -= fps
        screen.blit(self.image, (self.x, self.y))
 
    def update(self, action, dialogue):
        # Get images that are used to animate the given action
        flag = 0
        if action not in self.images or len(self.images[action]) == 0:
            print("Adding images")
            self.images[action] = [pygame.transform.scale(pygame.image.load(img) , (200,200)) for img in glob.glob("assets\\characters\\" + self.char + "\\" + action + "\\*.png")]
            print("Self.images = ", self.images)

        if self.index >= len(self.images[action]):
            flag = 1
            self.index = 0
            self.prev_action = action

        if not flag:
            self.image = self.images[action][self.index]
            self.index += 1

            self.movement_update(FPS if actions_movement[action] else 0)

            # # Say dialogue
            # if action == "say":
            #     self.play_dialogue(dialogue)
            #     print("dialogue played")
        else:
            self.prev_action = action
            self.play_prev_frame()
        

        print("self.index: ", self.index - 1)

        return flag

    def play_prev_frame(self):
        if self.prev_action is None:
            return
        
        self.image = self.images[self.prev_action][-1]

        print("PLAYING PREVIOUS ACTION ", self.prev_action, " OF ", self.char, " IMAGE ", self.image)

        self.movement_update(0)
 
def main():
    pygame.init()
    pygame.display.set_caption("Trace")

    char_objects = dict()

    for i, character in enumerate(characters):
        dir = list(dir_list.keys())[i % 2]
        char_objects[character] = MySprite(character, dir_list[dir] * ((i % 2) + 1), 480, dir)

    # my_group = pygame.sprite.Group(my_sprite)
    idle_char = []
    clock = pygame.time.Clock()


    return_val = 0
    for svo in svos:
        print(svo)
        character = svo[0]
        action = svo[1]
        dialogue = ""

        if(len(svo) == 3):
            dialogue = svo[2]
        
        flag = 0
        pygame.event.get()

        while not flag:
            screen.fill((0,0,0))        
            screen.blit(bg, (0, 0))

            flag = char_objects[character].update(action, dialogue)

            for char in char_objects:
                if char != character:
                    char_objects[char].play_prev_frame()

            pygame.display.update()
            clock.tick(FPS)

            if dialogue != "":
                char_objects[character].play_dialogue(dialogue)
                print("dialogue played")
                dialogue = ""

    pygame.quit()
 
if __name__ == '__main__':
    main()

('cat', 'walk')
Adding images
Self.images =  {'walk': [<Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>]}
self.index:  0
self.index:  1
self.index:  2
self.index:  3
self.index:  4
self.index:  5
self.index:  6
self.index:  7
self.index:  8
self.index:  9
PLAYING PREVIOUS ACTION  walk  OF  cat  IMAGE  <Surface(200x200x32 SW)>
self.index:  -1
('cat', 'jump')
Adding images
Self.images =  {'walk': [<Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>], 'jump': [<Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x

# NER

In [ ]:
import spacy 
import spacy.cli
# Download following once
#spacy.cli.download("en_core_web_sm")

In [ ]:
from spacy import displacy

nlp = spacy.load('en_core_web_sm')

C:\Users\shour\AppData\Roaming\Python\Python39\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.4.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
text = "Victoria was a major in CS. Victoria studies at MIT."
doc = nlp(text)

ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print(ents)
# now we use displaycy function on doc2
displacy.render(doc, style='ent', jupyter=True)

[('CS', 24, 26, 'ORG'), ('MIT', 48, 51, 'ORG')]


In [ ]:
from gtts import gTTS
  
# This module is imported so that we can 
# play the converted audio
import os
  
# The text that you want to convert to audio
mytext = 'Welcome to geeksforgeeks!'
  
# Language in which you want to convert
language = 'en'
  
# Passing the text and language to the engine, 
# here we have marked slow=False. Which tells 
# the module that the converted audio should 
# have a high speed
myobj = gTTS(text=mytext, lang=language, slow=False)
  
# Saving the converted audio in a mp3 file named
# welcome 
myobj.save("welcome.mp3")
  
# Playing the converted file
os.system("mpg321 welcome.mp3")

1

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

# object and subject constants
OBJECT_DEPS = {"dobj", "dative", "attr", "oprd"}
SUBJECT_DEPS = {"nsubj", "nsubjpass", "csubj", "agent", "expl"}
# tags that define wether the word is wh-
WH_WORDS = {"WP", "WP$", "WRB"}

# extract the subject, object and verb from the input
def extract_svo(doc):
    sub = []
    at = []
    ve = []
    for token in doc:
        # is this a verb?
        if token.pos_ == "VERB":
            ve.append(token.text)
        # is this the object?
        if token.dep_ in OBJECT_DEPS or token.head.dep_ in OBJECT_DEPS:
            at.append(token.text)
        # is this the subject?
        if token.dep_ in SUBJECT_DEPS or token.head.dep_ in SUBJECT_DEPS:
            sub.append(token.text)
    return " ".join(sub).strip().lower(), " ".join(ve).strip().lower(), " ".join(at).strip().lower()

# wether the doc is a question, as well as the wh-word if any
def is_question(doc):
    # is the first token a verb?
    if len(doc) > 0 and doc[0].pos_ == "VERB":
        return True, ""
    # go over all words
    for token in doc:
        # is it a wh- word?
        if token.tag_ in WH_WORDS:
            return True, token.text.lower()
    return False, ""

# gather the user input and gather the info
doc = nlp("A cat is walking on a sunny day. A cat jumped. A cat was talking with a human and died. Dog is walking in the opposite direction. Dog ran.")
print(doc)
# print out the pos and deps
for token in doc:
    print("Token {} POS: {}, dep: {}".format(token.text, token.pos_, token.dep_))

# get the input information
filtered_subject, verb, attribute = extract_svo(doc)
question, wh_word = is_question(doc)
print("svo:, subject: {}, verb: {}, attribute: {}, question: {}, wh_word: {}".format(filtered_subject, verb, attribute, question, wh_word))

C:\Users\shour\AppData\Roaming\Python\Python39\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.4.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


A cat is walking on a sunny day. A cat jumped. A cat was talking with a human and died. Dog is walking in the opposite direction. Dog ran.
Token A POS: DET, dep: det
Token cat POS: NOUN, dep: nsubj
Token is POS: AUX, dep: aux
Token walking POS: VERB, dep: ROOT
Token on POS: ADP, dep: prep
Token a POS: DET, dep: det
Token sunny POS: ADJ, dep: amod
Token day POS: NOUN, dep: pobj
Token . POS: PUNCT, dep: punct
Token A POS: DET, dep: det
Token cat POS: NOUN, dep: nsubj
Token jumped POS: VERB, dep: ROOT
Token . POS: PUNCT, dep: punct
Token A POS: DET, dep: det
Token cat POS: NOUN, dep: nsubj
Token was POS: AUX, dep: aux
Token talking POS: VERB, dep: ROOT
Token with POS: ADP, dep: prep
Token a POS: DET, dep: det
Token human POS: NOUN, dep: pobj
Token and POS: CCONJ, dep: cc
Token died POS: VERB, dep: conj
Token . POS: PUNCT, dep: punct
Token Dog POS: PROPN, dep: nsubj
Token is POS: AUX, dep: aux
Token walking POS: VERB, dep: ROOT
Token in POS: ADP, dep: prep
Token the POS: DET, dep: det
Toke